# Query Data with AWS Data Wrangler
* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

AWS Data Wrangler is an open-source Python package that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

In [1]:
!pip install -q awswrangler==1.2.0

In [2]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [3]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

dataset (bool) – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [4]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                     columns=['star_rating', 'product_category', 'review_body'],
                                     filters=[("product_category", "=", "Digital_Software")],
                                     dataset=True)
df_parquet_results.shape

(102084, 3)

In [5]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,5,"It was easy to install,easy to update either m...",Digital_Software
1,2,Terrible software. The only reason to get it i...,Digital_Software
2,3,Liked my Quicken 2007 better. 2014 too busy fo...,Digital_Software
3,3,State will not e-file no mater what. Huge disa...,Digital_Software
4,4,Nice. Got a good deal right around the time wh...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [6]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                filters=[("product_category", "=", "Digital_Software")],
                                dataset=True,
                                chunked=True)

In [7]:
print(next(chunk_iter))

     star_rating                                        review_body  \
0              5  It was easy to install,easy to update either m...   
1              2  Terrible software. The only reason to get it i...   
2              3  Liked my Quicken 2007 better. 2014 too busy fo...   
3              3  State will not e-file no mater what. Huge disa...   
4              4  Nice. Got a good deal right around the time wh...   
..           ...                                                ...   
758            2  My frustrations with Quickbooks continue to be...   
759            1  This is ridiculous. I wanted to spend my money...   
760            4  I've used TurboTax for years with great result...   
761            3  I have been using AVG for 2 or 3 years. I had ...   
762            5  I have been using Norton for 7 years and after...   

     product_category  
0    Digital_Software  
1    Digital_Software  
2    Digital_Software  
3    Digital_Software  
4    Digital_Software  
.. 

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [8]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'

In [9]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [10]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database=database_name
)

CPU times: user 282 ms, sys: 19 ms, total: 301 ms
Wall time: 7.65 s


In [11]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12892960,R2XR0UT1T6PLK2,B00U1UEPEQ,636727185,Corel Parent,Digital_Software,4,1,1,N,Y,ok fro $$,Not as good as other video editing software bu...,2015-08-26
1,US,38066706,RXHDB0XFQ4ZNG,B00B1TGUMG,284323980,Microsoft Office Home and Student 2013 (1PC/1U...,Digital_Software,3,2,2,N,Y,Buy if you must,"It's MS, what can you say? Some of the feature...",2015-08-26
2,US,44064300,RB73SNLMTWW61,B00B1TGUMG,284323980,Microsoft Office Home and Student 2013 (1PC/1U...,Digital_Software,5,0,2,N,Y,Five Stars,works!,2015-08-26
3,US,40284469,R2AMUKDZL5F8AY,B00LU2XOLO,325816739,"Kaspersky Internet Security 2015 3 User, 1 Year",Digital_Software,3,0,0,N,Y,Three Stars,can register only in local..<br />here is cann...,2015-08-26
4,US,1969985,R1066MVAFC477L,B00JPRUDFQ,23010115,WordPerfect Office X7 Home and Student,Digital_Software,3,0,0,N,Y,Three Stars,It's good,2015-08-26


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.

In [12]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

CPU times: user 193 ms, sys: 17.4 ms, total: 210 ms
Wall time: 4.25 s


In [13]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    21269168   RSH1OZ87OYK92  B013PURRZW      603406193   
1             US      133437  R1WFOQ3N9BO65I  B00F4CEHNK      341969535   
2             US    45765011   R3YOOS71KM5M9  B00DNHLFQA      951665344   
3             US      113118  R3R14UATT3OUFU  B004RMK5QG      395682204   
4             US    22151364   RV2W9SGDNQA2C  B00G9BNLQE      640460561   
...          ...         ...             ...         ...            ...   
4995          US    16124142  R300681GTFNO5L  B011S6DBTG      402709566   
4996          US     1688095  R3F8QIC548OKSB  B010KYDNDG      835376637   
4997          US    50083410  R1JZ3PA71KYCK2  B004RMK4BC      384246568   
4998          US    13538939   RCRSJ8DOYHUXC  B0030T1AK2      329510149   
4999          US    53090127  R2CLUUCAVBHN1F  B00P9QGNEM      577605087   

                                          product_title     product_category  \
0                 M

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();